We test the SqLinOp wrapper for computing eigenvalues of implicitly represented square linear operators.

It seems that the power method code eigvals_power from the *IterativeSolvers* package is broken?

But eigvals_lanczos might be working, however it's not clear how to get only the largest eig, since a call for one eig seems to sometime produce a smaller eig?

It's probably better just to go for the *eigs* function from standard julia lin alg.

*Tentatively* it seems that using :LM option for eigs gives the eigenvalue that's largest in magnitude, which is good enough for our purposes?

In [1]:
using IterativeSolvers

In [2]:
include("../src/sqLinOpWrapper.jl")

testId (generic function with 1 method)

In [5]:
M = testId(10)

SqLinOp{Float64,Int64}(true,1.0,10,(anonymous function))

In [85]:
@time M*[1.0]

  0.000005 seconds (5 allocations: 240 bytes)


1-element Array{Float64,1}:
 1.0

Testing with eigvals_power
--------------------------

In [86]:
@time eigvals_power(M)[1].val

  0.000045 seconds (31 allocations: 1.641 KB)


1.0

In [87]:
M2fn = x -> [[-2 0]; [0 1]]*x

(anonymous function)

In [88]:
M2Op = SqLinOp(true,1.0,2,M2fn)

SqLinOp{Float64,Int64}(true,1.0,2,(anonymous function))

In [89]:
@time eigvals_power(M2Op)[1].val

  0.002496 seconds (95 allocations: 4.522 KB)


-1.8434516894686996

Wow, that doesn't look too good?

In [126]:
M3fn = x -> [[2 0]; [0 1]]*x; M3Op = SqLinOp(true,1.0,2,M3fn)

SqLinOp{Float64,Int64}(true,1.0,2,(anonymous function))

In [127]:
@time eigvals_power(M3Op)[1].val

  0.002420 seconds (95 allocations: 4.520 KB)


1.6237150104497309

Again, that looks bad.

In [128]:
@time eigvals_lanczos(M2Op,1)[1]

  0.000160 seconds (207 allocations: 10.594 KB)


1-element Array{Float64,1}:
 1.0

That looks ok?

In [129]:
@time eigvals_lanczos(M3Op,2)[1]

  0.000166 seconds (207 allocations: 10.594 KB)


2-element Array{Float64,1}:
 1.0
 2.0

In [130]:
@time eigvals_lanczos(M3Op,1)[1]

  0.000194 seconds (207 allocations: 10.594 KB)


1-element Array{Float64,1}:
 1.0

Hmm, I'm not sure how to get out only the larger eigenvalue of M3Op.

Testing with eigs
-----------------

In [116]:
@time eigs(M;nev=1,which=:LM)[1][1]

  0.000156 seconds (121 allocations: 9.484 KB)


1.0

In [117]:
@time eigs(M2Op;nev=1,which=:LM)[1][1]

  0.000182 seconds (121 allocations: 6.266 KB)


-1.9999999999999998

In [119]:
@time eigs(M2Op;nev=1,which=:SM)[1][1]

LoadError: LoadError: MethodError: `factorize` has no method matching factorize(::SqLinOp{Float64,Int64})
while loading In[119], in expression starting on line 155

In [120]:
@time eigs(M3Op;nev=1,which=:LM)[1][1]

  0.000205 seconds (121 allocations: 6.266 KB)


2.0

In [123]:
d = randn(10)

10-element Array{Float64,1}:
  0.87926 
  0.193346
  0.56746 
 -0.723257
 -2.0645  
  1.11591 
 -0.247983
 -0.206008
 -0.68942 
  0.359892

In [125]:
M4fn = x -> d.*x; M4Op = SqLinOp(true,1.0,10,M4fn)

SqLinOp{Float64,Int64}(true,1.0,10,(anonymous function))

In [134]:
@time eigs(M4Op;nev=1,which=:LM)[1][1]

  0.000246 seconds (201 allocations: 13.391 KB)


-2.064504786327293

Ok, it seems we get the eig that's largest in magnitude.

Testing with our LDL
--------------------

In [168]:
using Laplacians

In [169]:
push!(LOAD_PATH,"../Laplacians.jl/src")

5-element Array{ByteString,1}:
 "/usr/local/Cellar/julia/0.4.5/local/share/julia/site/v0.4"
 "/usr/local/Cellar/julia/0.4.5/share/julia/site/v0.4"      
 "../Laplacians.jl/src"                                     
 "../Laplacians.jl/src"                                     
 "../Laplacians.jl/src"                                     

In [1]:
include("../src/samplingSolver.jl")

tomatrix (generic function with 1 method)

In [3]:
A = chimera(10,1);

In [4]:
n = A.n

10

In [13]:
f,gOp,U,d = buildSolver(A);

In [6]:
fOp = SqLinOp(true,1.0,n,f)

SqLinOp{Float64,Int64}(true,1.0,10,(anonymous function))

In [8]:
@time eigs(fOp;nev=1,which=:LM)[1][1]

  0.000609 seconds (491 allocations: 55.266 KB)


2.1604506204523295

In [9]:
L = lap(A);

In [10]:
norm(Symmetric(L)-L)

0.0

In [11]:
eigs(Symmetric(pinv(full(L)));nev=1,which=:LM)[1][1]

2.1888891924987695

looks reasonable?

In [15]:
@time eigs(gOp;nev=1,which=:LM)[1][1]

  0.000626 seconds (781 allocations: 59.797 KB)


-0.04724310116513397

In [16]:
@time eigs(gOp)[1]

  0.002700 seconds (1.37 k allocations: 93.650 KB)


6-element Array{Float64,1}:
 -0.0472431 
  0.0280675 
 -0.00772168
  0.00672949
  0.00456993
  6.61894e-5

In [17]:
norm(full(L - U * spdiagm(d) * U'))

0.09518690709748781

In [18]:
dih = zeros(n); dih[1:(n-1)] = d[1:(n-1)].^(-1/2); dih'

1x10 Array{Float64,2}:
 0.57735  0.609961  0.617282  0.707107  …  0.887277  0.965056  1.1522  0.0

In [19]:
Ui = full(U)^(-1);

In [20]:
Papx = diagm(dih)*Ui*L*Ui'*diagm(dih);

In [21]:
P = eye(n); P[n,n] = 0;

In [23]:
@time eigs(Symmetric(Papx-P);nev=1,which=:LM)[1][1]

  0.016323 seconds (5.19 k allocations: 275.069 KB, 35.04% gc time)


-0.04724310116513395

> eigs(gOp;nev=1,which=:LM)[1][1]

agrees with

> eigs(Symmetric(Papx-P);nev=1,which=:LM)[1][1]

so I think the implicit representation is working.

How to do the check
-------------------

In [47]:
f,gOp,U,d = buildSolver(A);

In [49]:
abs(checkError(gOp)) < 0.5

true